# IUM projekt - Zniżki dla klientów

## Zadanie
- Zinterpretować: “Niektórzy  klienci  nie  mogą  zdecydować  się  na zakup oglądając  produkt  na  stronie – ale pewnie  gdybyśmy  wiedzieli,  które  to  są  osoby,  to odpowiednia zniżka  skłoniłaby ich do zakupu."  
- Analiza danych.  
- Doprecyzowanie, przeanalizowanie problemu i wdrozenie aplikacji bazującej na mikroserwisach.  

## Problem biznesowy
Stworzenie systemu, który umożliwi zaproponowanie klientom niezdecydowanym, zniżki takiej, która zachęci ich do zakupu, ale równocześnie bedzie jak najmniejsza.  
Istnieje kilka interpretacji zadania:  
1. Model ma mówić czy klient jest niezdecydowany
2. Model ma mówić czy klient jest niezdecydowany i jakoś optymalizować stawkę zniżki
3. Model ma mówić czy opłaca się zaproponować zniżkę klientowi
4. Model ma wyznaczać wartośc zniżki którą powinno się zaproponować każdemu klientówi na podstawie jego historii

## Zadania modelowania
**Zakładając interpretację nr.2**
- zadanie klasyfikacji ( klasyfikacja klientów w klasy o różnym stopniu niezdecydowania )
- zadanie regresji ( znalezienie takiej znizki, która spowoduje, że klient zdecyduje się na zakup, ale firma zarobi jak najwięcej ) - do doprecyzowania, może wystarczy przypisać daną zniżkę do danej klasy

## Założenia
* Będziemy na zbiorze testowym sprawdzać czy dobrze przewidujemy, że dany klient jest niezdecydowany. To znaczy, że sprawdzamy historie klienta i jeżeli wyświetlał produkty a po zniżce dokonał zakupu, to oznacza, że był niezdecydowany.
* Odpowiedna zniżka to taka, która zachęci klienta do zakupu. Przy czym musi ona przynieść jak najwiekszy zysk firmie.
* Ważnym indykatorem, który może sugerować, że klient jest niezdecydowany to taki przypadek klienta, którego ostatnich 5(można zmienić) wyświetleń produktu należy do tej samej kategorii. Dodatkowo jeżeli ostatne 3 wyświetlenia dotyczą tego samego produktu to jest bardzo zainteresowany, ale nadal niezdecydowany.
* Zakładamy, że lepiej, dać klientowi zniżkę nawet jeżeli jest zdecydowany, niż nie wykryć, że klient był niezdecydowany.

## Kryteria sukcesu
- Stworzenie systemu, który przyniesie jak największy zysk firmie.  
Składa się na to zmotywowanie do zakupu jak największej ilości klientów, ale nie kosztem straty wynikającej z za dużej zniżki.  
- Chcemy, żeby nasz model miał jak największy *Recall*, żeby z jak największym prawdopodobieństwem wykrywał potencjalnego niezdecydowanego.  
- W ostatniej fazie wykonać testy online na klientach i sprawdzić czy nasz system przynosi zysk.  


## Poniżej wstępna analiza danych

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import random
import numpy as np
import pandas as pd

In [2]:
users = pd.read_json(path_or_buf='data/users.jsonl', lines=True)
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  200 non-null    int64 
 1   name     200 non-null    object
 2   city     200 non-null    object
 3   street   200 non-null    object
dtypes: int64(1), object(3)
memory usage: 6.4+ KB


In [3]:
deliveries = pd.read_json(path_or_buf='data/deliveries.jsonl', lines=True)
deliveries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3117 entries, 0 to 3116
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   purchase_id         3117 non-null   int64  
 1   purchase_timestamp  3117 non-null   object 
 2   delivery_timestamp  2962 non-null   object 
 3   delivery_company    2963 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 97.5+ KB


In [4]:
sessions = pd.read_json(path_or_buf='data/sessions.jsonl', lines=True)
sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56883 entries, 0 to 56882
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   session_id        56883 non-null  int64         
 1   timestamp         56883 non-null  datetime64[ns]
 2   user_id           54075 non-null  float64       
 3   product_id        54048 non-null  float64       
 4   event_type        56883 non-null  object        
 5   offered_discount  56883 non-null  int64         
 6   purchase_id       3117 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(1)
memory usage: 3.0+ MB


"RangeIndex: 56883 entries, 0 to 56882  
6   purchase_id       3117 non-null   float64 "  
  
Mamy więc dużo braków w purchase_id.

In [5]:
products = pd.read_json(path_or_buf='data/products.jsonl', lines=True)
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319 entries, 0 to 318
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     319 non-null    int64  
 1   product_name   319 non-null    object 
 2   category_path  319 non-null    object 
 3   price          319 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 10.1+ KB
